# Data Pre-processing & Feature Selection

In [ ]:
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available
# on an “as is,” “as available,” and “with all faults” basis, solely for
# illustrative purposes, and without warranty or representation of any kind.
# This solution is experimental, unsupported and provided solely for your
# convenience. Your use of it is subject to your agreements with Google, as
# applicable, and may constitute a beta feature as defined under those
# agreements.  To the extent that you make any data available to Google in
# connection with your use of the solution, you represent and warrant that you
# have all necessary and appropriate rights, consents and permissions to permit
# Google to use and process that data.  By using any portion of this solution,
# you acknowledge, assume and accept all risks, known and unknown, associated
# with its usage, including with respect to your deployment of any portion of
# this solution in your systems, or usage in connection with your business,
# if at all.
###########################################################################

## 0) Dependencies

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name

In [ ]:
# Google credentials authentication libraries
from google.colab import auth
auth.authenticate_user()

import sys
import warnings

# data processing libraries
import numpy as np
from numpy.core.numeric import NaN
import datetime
import pandas as pd
import pandas_gbq

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
!pip install boruta #boruta for feature selection
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

# modeling and metrics
from scipy.optimize import least_squares
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm


import itertools
from scipy.stats.stats import pearsonr

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5] #change size of plot
import seaborn as sns
import plotly.express as px

# BigQuery Magics
'''
BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI
'''

from google.cloud import bigquery
from google.cloud.bigquery import magics

magics.context.project = project_name

client = bigquery.Client(project=magics.context.project)

## 1) Import dataset
#####Only one of the following sections should be executed, either import from CSV, BigQuery, or Google Sheets.

##Import Original Data from BigQuery (Option #1)

In [ ]:
#@title
auth.authenticate_user()
bigquery.USE_LEGACY_SQL = False

magics.context.project = project_name
client = bigquery.Client(project=magics.context.project)
%load_ext google.cloud.bigquery
project_id = project_name
dataset_name = '' #@param {type:"string"}
table_name = '' #@param {type:"string"}
query = 'select * from `' +  dataset_name + '.'+ table_name + '`'
df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
df.head()

## Upload Original Data CSV (Option #2)

In [ ]:
#update file name
file_name = ''
df = pd.read_csv(file_name)
df.fillna(0, inplace=True)
df.head()

## Import Data from Sheets (Option #3)

In [ ]:
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()
sheet_name = '' #@param {type:"string"}
worksheet = gc.open(sheet_name).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
original_data_import = pd.DataFrame.from_records(rows)
df = original_data_import.rename(
    columns=original_data_import.iloc[0]).drop(
        original_data_import.index[0]).reset_index(drop=True).astype('float')
df.head()

The first step is to remove variables that won't be used in the model. In this example, we remove columns like geo which is consistent across the dataset and aggregated media such as total clicks across DSPs.


In [ ]:
df.drop(columns = ['geo','x1','x2','x8','x18','x19','x20','x21','x22','x23','x24','x25'], inplace = True)

In [ ]:
len(df.columns)

In [ ]:
df.describe()

In [ ]:
#Set the date as index
date_col = "date" #@param {type:"string"}
df = df.sort_values(date_col).set_index(date_col)

In [ ]:
'''If using daily data in the final model, set to True.
If data is already at the weekly level OR if you'd like to
roll daily data up to weekly level, leave as false
'''
is_daily_data = True #@param {type:"boolean"}

Option to aggregate daily data to weekly data

In [ ]:
# Uncomment and run code below if you'd like to roll daily data up to weekly level.
'''
if is_daily_data == False:
  df = df.resample('7D').sum()
  '''

In [ ]:
df.head()

## 2) Data Cleaning

### 2.1) Check for missing data and impute

Check the amount of of missing values (% of total column) in the data and sort by
highest to lowest.

In [ ]:
missing_values = 100*df.isnull().sum()/len(df)
missing_values.sort_values(ascending = False)

If there are any NAs in the data that should be zeros, replace those data
points with zero.

In [ ]:
df.fillna(0, inplace = True)

## 3) Define Y (KPI column) and create initial feature set

In [ ]:
#Input column names for Y (ex: "new_accounts" or "sales")
kpi_col = "y1" #@param {type:"string"}
target_variable = df[kpi_col] #y variable

In [ ]:
# Create a dataframe for features (all variables except date and kpi) x variables
featureset_df = df[df.columns[df.columns != date_col]]
featureset_df = df[df.columns[df.columns != kpi_col]]
# Converting data to float
featureset_df = featureset_df.astype(float, copy=False)
featureset_df.head()

## 4) Visualize Series

Optional:

Visualizing each series is useful to better understand the underlying distribution of the data. This allows for examination of outliers.

Understanding the distribution of the underlying data can also inform prior parameterization in bayesian modeling approaches later on.

In [ ]:
for i in range(2,len(featureset_df.columns)):
  plt.figure()
  sns.kdeplot(featureset_df[featureset_df.columns[i]], label = featureset_df.
              columns[i], fill = True)

## 5) Feature Creation

### 5.1) Check for Seasonality and add Flag

View the target variable as a time series plot and identify periods where data peaks.

We also add flags for periods of peak seasonality such as Q2, Q3, and major winter holidays.

Note: Please update seasonality with regards to your specific data requirements.


In [ ]:
fig = px.line(df[kpi_col])
fig.show()

In [ ]:
featureset_df['Is_Q2Q3'] = (df.index.get_level_values(0).month == 4).astype(int) |
  (df.index.get_level_values(0).month == 5).astype(int) |
  (df.index.get_level_values(0).month == 6).astype(int) |
  (df.index.get_level_values(0).month == 7).astype(int) |
  (df.index.get_level_values(0).month == 8).astype(int) |
  (df.index.get_level_values(0).month == 9).astype(int)

featureset_df['Is_Holiday'] = ((df.index == '2017-11-17') |
                               (df.index == '2017-12-22') |
                               (df.index == '2018-11-16') |
                               (df.index == '2018-12-21') |
                               (df.index == '') | (df.index == ''))

### 5.2) Lag, Diminishing Returns, Adstock

We'll need to transform the raw data by applying Adstock to have it most accurately predict the target variable.

**Adstock:** Applies an infinite lag that decreases its weight as time passes.
Media effect on sales may lag behind the original exposure and extend several weeks. More info can be found here: https://en.wikipedia.org/wiki/Advertising_adstock


Adstock takes 3 parameters (L, P, D):

L: length of the media effect

P: peak/delay of the media effect, how many days/weeks it’s lagging behind first exposure

D: decay/retention rate of the media channel, concentration of the effect

**First, split the df into two different dataframes:**

1. Features that **don't** need to be transformed
      - Examples are:
          - date
          - target variable
          - control variables (seasonality, promotions, etc.)

2. Features that **do** need to be transformed
      -  Paid media tactics
      -  Any other feature where there is some sort of delayed response with the target variable



In [ ]:
'''
Variables that do not need to be transformed (i.e. Target variable + controls)

Note: In this example case, almost all of the features in the featureset_df
are media features.As more dummy variables or other control variables are added,
the user will need to specify which columns should be transformed
'''

untransformed_df = pd.concat([target_variable,
                              featureset_df[['Is_Q2Q3','Is_Holiday']]],
                             axis = 1)

In [ ]:
transformed_df_partial = featureset_df.loc[:,~featureset_df.columns.isin(
    untransformed_df)]

#### 5.2.1) Create the transformation functions

This below function creates all permutations of the Adstock parameters and then calculates the correlation between each variable and the Y variable. Returns the top 3 highest correlated features.

L: fixed value. 8 if data is weekly and 21 if data is daily

P: multiple values between 1 - 7

D: multiple values between 0 - 0.48

Please note, the parameters can be adjusted in the code below.

In [ ]:
def createTransformations(df1, df2):
  columns = df2.columns
  sales = df1[[kpi_col]]
  all_data = []
  for col in columns:
    newdfcolumns = []
    for l in np.linspace(8 if is_daily_data else 21,
                         8 if is_daily_data else 21, 1):
      for p in np.round(np.linspace(1, 7, 7),2):
        for d in np.round(np.linspace(0, 0.48, 9),2):
          newdf = pd.DataFrame()
          newdf[f'{col}l{l}p{p}d{d}'.replace('.','_')] = Transformation(
              df2[col], col, int(2), p, d)
          newdfcolumns.append(newdf)
    concat_new_df = pd.concat(newdfcolumns, axis=1)
    corr_df = pd.concat([sales, concat_new_df], axis=1)
    corr = abs(corr_df.corr()).sort_values(kpi_col, ascending=False)
    new_vals= corr.iloc[1:4 , 0:1].index.tolist()
    data = concat_new_df[new_vals]
    all_data.append(data)
  final_data = pd.concat(all_data,axis=1)

  return final_data

In [ ]:
#This function creates every combination of Adstock

def Transformation(data, x, L, P, D):
    x_orig = data
    x_0 = np.append(np.zeros(L-1), x_orig)
    weights = np.zeros(L)
    for l in range(L):
      weight = D**((l-P)**2.0)
      weights[L-1-l] = weight
    adstocked_x = []
    for i in range(L-1, len(x_0)):
      x_array = x_0[i-L+1:i+1]
      xi = sum(x_array * weights)/sum(weights)
      adstocked_x.append(xi)
    return adstocked_x

#### 5.2.1) Execute the transformations

In [ ]:
transformed_df_partial = transformed_df_partial.sort_values(date_col)
sys.setrecursionlimit(len(transformed_df_partial.index)+100)

with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  transformed_df_partial = createTransformations(untransformed_df,
                                                 transformed_df_partial)
transformed_df_partial.head()

### 6) Feature Scaling

6.1) Train/Test Split & Feature Scaling

Please specify the test dataset size below.

Completing train/test split prior to standardizing and feature selection ensures no data leakate.

**Note:** use the same test % size when building the model.

In [ ]:
test_size_percentage = 0.1 #@param {type:"slider", min:0, max:0.3, step:0.01}

The default method of standardization utilizes Standard Scaler, which takes in
input data and transforms so that the output has mean 0 and standard deviation of 1
across all features.


Alternative methods of feature scaling include square-root transformation,
de-meaning, natural log transformations, Min-Max Scalers, or normalization

In [ ]:
test_size = round(len(transformed_df_partial.index)*test_size_percentage)

untransformed_df.reset_index(inplace=True, drop=True)
transformed_df = pd.concat(
    [transformed_df_partial, untransformed_df[untransformed_df.columns[
        untransformed_df.columns != kpi_col]]], axis = 1)
dataset_split = train_test_split(transformed_df,
                                 untransformed_df[[kpi_col]],
                                 test_size=test_size, shuffle=False)

transformed_df_train = dataset_split[0]
transformed_df_test = dataset_split[1]
transformed_df_train_target = dataset_split[2]
transformed_df_test_target = dataset_split[3]
train_scaler = StandardScaler()
standardized_array_train = train_scaler.fit_transform(transformed_df_train)
standardized_array_test = train_scaler.transform(transformed_df_test)
standardized_df_test = pd.DataFrame(
    standardized_array_test, columns = transformed_df_test.columns)
standardized_df_train = pd.DataFrame(
    standardized_array_train, columns = transformed_df_train.columns)
standardized_df_test.reset_index(inplace=True, drop=True)
standardized_df_train.reset_index(inplace=True, drop=True)
transformed_df_test_target.reset_index(inplace=True, drop=True)
transformed_df_train_target.reset_index(inplace=True, drop=True)

trimmed_transformed_standardized_train_w_target = pd.concat(
    [transformed_df_train_target, standardized_df_train], axis=1)
trimmed_transformed_standardized_train_w_o_target = standardized_df_train
trimmed_transformed_standardized_test_w_target = pd.concat(
    [transformed_df_test_target, standardized_df_test], axis=1)
trimmed_transformed_standardized_test_w_o_target = standardized_df_test

Option to review visuals of the data. After the data is standardized the distributions may take on a more normal shape.

In [ ]:
'''

for i in range(0,len(trimmed_transformed_standardized_train_w_o_target.columns)):
  plt.figure()
  sns.kdeplot(trimmed_transformed_standardized_train_w_o_target[
      trimmed_transformed_standardized_train_w_o_target.columns[i]], label =
      trimmed_transformed_standardized_train_w_o_target.columns[i], shade = True)

'''

## 7) Feature Selection

For feature selection we employ the Boruta algorithm.[(More information here)](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a
)

This algorithm will tell you the rank of each feature and whether or not to keep a varaible in the model (i.e. Keep  = True/False). The goal of RBA is to optimize across all paid digital media tactics, therefore select the top ranking feature for each group of features (whether or not the algorithm tells you to keep the feature).


In [ ]:
# Specifiying the target and x variables
num_train_samples = transformed_df_train_target.shape[0]

y = transformed_df_train_target.to_numpy().reshape(num_train_samples,)
x = trimmed_transformed_standardized_train_w_o_target

In [ ]:
# define random forest classifier
forest = RandomForestRegressor(n_jobs=-1, max_depth=5)
forest.fit(x, y)

In [ ]:
with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  # define Boruta feature selection method
  feat_selector = BorutaPy(
      forest, n_estimators='auto', verbose=0, random_state=1)

  # find all relevant features
  feat_selector.fit(np.array(x), np.array(y))

  # check selected features
  feat_selector.support_

  # check ranking of features
  feat_selector.ranking_

  # call transform() on X to filter it down to selected features
  X_filtered = feat_selector.transform(np.array(x))

In [ ]:
#Select the top ranking variable for each group of variables.
feature_ranks = list(zip(x.columns,
                         feat_selector.ranking_,
                         feat_selector.support_))

# iterate through and print out the results
for feat in feature_ranks:
    print('{:<25}, Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

Function to reduce the overall dataset to the lowest rank feature from the Boruta output, and save to a dataframe

In [ ]:
def stripQualifiers(column, leading=True):
  retString = column
  if retString.startswith('cost_') and leading:
    retString = retString.split('cost_')[1]
  if retString.startswith('Paid_') and leading:
    retString = retString.split('Paid_')[1]
  try:
    int(retString[::-1][retString[::-1].find('l')-1])
    retString = retString[::-1][retString[::-1].find('l')+1:][::-1]
    return retString
  except:
    return retString

keepers = {}
for feat in feature_ranks:
    #print('{:<25}, Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
    featureName = stripQualifiers(feat[0], True)
    if keepers.get(featureName) is None:
      keepers[featureName] = {'fullColumnName': feat[0], 'rank': feat[1]}
    else:
      if keepers[featureName]['rank'] > feat[1]:
        keepers[featureName] = {'fullColumnName': feat[0], 'rank': feat[1]}

selected_featureset_df = trimmed_transformed_standardized_train_w_o_target.drop(
    columns=[col for col in trimmed_transformed_standardized_train_w_o_target
             if col not in [keepers[i]['fullColumnName'] for i in keepers.keys()]])

selected_featureset_df.head()

## 8) Handle Multicollinearity (reduce feature set)

1. Print a correlation heatmap to visualize correlations across feature set
2. Run variance inflation factor analysis and output results to flag multicollinearity above specified threshold

In [ ]:
correl = selected_featureset_df.corr()

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(correl)

# using the upper triangle matrix as mask
sns.heatmap(correl, mask=matrix)

Run VIF analysis and flag values greater than 10.

Industry best practice flags values above 10 as an extreme violation of regression model assumptions. [(Reference)](https://en.wikipedia.org/wiki/Variance_inflation_factor)


In [ ]:
vif = add_constant(selected_featureset_df)

# loop to calculate the VIF for each X
vif = pd.Series([variance_inflation_factor(vif.values, i)
      for i in range(vif.shape[1])],
      index=vif.columns)

In [ ]:
# processing to output VIF results as a dataframe
vif_df=vif.to_frame().reset_index()

vif_df.columns = ['feature', 'vif']
vif_df=vif_df.replace([np.inf], np.nan)
vif_df=vif_df.fillna(0).sort_values(by="vif", ascending=False)

In [ ]:
vif_df.reset_index(inplace = True)
vif_df

Drop the highest VIF features and print the high collinearity columns in a list

In [ ]:
high_collinearity_columns = vif_df.feature[vif_df['vif'] >= 10].to_list()
high_collinearity_columns

Drop 1 variable at a time (start with the highest VIF) and re-run the VIF cell to re-check multicollinearity. This will allow the user to preserve as many features in the model as possible.

In [ ]:
cols_to_drop = []
while vif_df.vif[1] >= 10:
  if vif_df.vif[1] >= 10:
    cols_to_drop.append(vif_df.feature[1])
    selected_featureset_df.drop(columns = vif_df.feature[1],inplace = True)
    vif = add_constant(selected_featureset_df)
  # loop to calculate the VIF for each X
    vif = pd.Series([variance_inflation_factor(vif.values, i)
    for i in range(vif.shape[1])], index=vif.columns)
    # processing to output VIF results as a dataframe
    vif_df=vif.to_frame().reset_index()
    vif_df.columns = ['feature', 'vif']
    vif_df=vif_df.replace([np.inf], np.nan)
    vif_df=vif_df.fillna(0).sort_values(by="vif", ascending=False)
    vif_df.reset_index(inplace = True)

In [ ]:
cols_to_drop

In [ ]:
selected_featureset_df.columns

In [ ]:
len(selected_featureset_df.columns)

## 9) Export Final Dataset

### 9.1) Export data for model building


In [ ]:
final_df = pd.concat([trimmed_transformed_standardized_train_w_o_target,
                      trimmed_transformed_standardized_test_w_o_target])
final_df = final_df[final_df.columns[final_df.columns.isin(
    selected_featureset_df.columns)]]
final_df.reset_index(inplace=True, drop=True)
final_df[kpi_col] = target_variable.reset_index()[kpi_col]

In [ ]:
final_df[date_col] = df.index #add back in the date as a separate column from the index
final_df[date_col] = final_df[date_col].astype('string')

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

In [ ]:
destination_project_id = "" #@param {type:"string"}
destination_dataset = "RBA_demo" #@param {type:"string"}
destination_table = "cleaned_data" #@param {type:"string"}
dataset_table = destination_dataset+"."+destination_table

final_df.to_gbq(dataset_table,
                 destination_project_id,
                 chunksize=None,
                 if_exists='replace'
                 )

### 9.2) Prepare the data for optimization tool

The budget optimization tool requires the model features in their original un-transformed state as an input. The following function pulls the names of the required columns and collects the data from the relevant dataframes.

In [ ]:
def parse_final_column_features(columns):
  final = []
  for col in columns:
    if any(char.isdigit() for char in col):
      splitList = col[::-1].split('l', 1)
      parsed = (splitList[1] if len(splitList) > 1 else splitList[0])
      final.append(parsed[::-1])
    else:
      final.append(col)
  return final

In [ ]:
df[df.columns[df.columns.isin(parse_final_column_features(selected_featureset_df.columns))]].head()

In [ ]:
optimizer_df = df[df.columns[df.columns.isin(parse_final_column_features(
    selected_featureset_df.columns))]]
optimizer_df.reset_index(inplace=True)
optimizer_df[date_col] = optimizer_df[date_col].astype('string')
target_variable.reset_index(inplace=True, drop=True)
optimizer_df[kpi_col] = target_variable

In [ ]:
destination_project_id = "" #@param {type:"string"}
destination_dataset = "RBA_demo" #@param {type:"string"}
destination_table = "optimizer_data" #@param {type:"string"}
dataset_table = destination_dataset+"."+destination_table

optimizer_df.to_gbq(dataset_table,
                 destination_project_id,
                 chunksize=None,
                 if_exists='replace'
                 )